In [1]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

import os
os.getcwd()
os.chdir('/Users/fizz/Document/Notes/Python/codes')

# Exception Coding Details

For reference, here's the general and most complete format in Python 3.X:

In [ ]:
try:
    statements
except name1:
    statements
except (name2, name3):
    statements            # Run if any of these exceptions occur
except name4 as var:
    statements            # Run if name4 is raised, assign instance raised to var
except:
    statements            # Run for all other exceptions raised
else:
    statements            # Run if no exception was raised during try block
finally:
    statements            # Always perform this block on exit

* Formally, there may be any number of __except__ clauses, but you can code __else__ only if there is at least one __except__, and there can be only one __else__ and one __finally__.

* __Exception__ has almost the same effect as an empty __except__, but ignores exceptions related to system exits.

No matter what's happened previously, the *finally-block* is executed once the main action block is complete and any raised exceptions have been handled. In fact, the code in the *finally-block* will be run even if there is an error in an exception handler or the *else-block* and a new exception is raised.  
The net effect is that the __finally__ is always run, regardless of whether:  
* An exception occurred in the main action and was handled
* An exception occurred in the main action and was not handled
* No exception occurred in the main action
* A new exception was triggered in one of the handlers

In [ ]:
raise instance      # Raise instance of class
raise class          # Make and raise instance of class: makes an instance
raise                  # Reraise the most recent exception

In [ ]:
exc = IndexError()
raise exc

excs = [IndexError, TypeError]
raise excs[0]

In [ ]:
class myExc(Exception): pass
raise myExc('spam')

try:
    ...
except MyExc as X:
    print(X.args)

Python 3.X localizes the exception reference name to the __except__ block - the variable is not available after the block exits, much like a temporary loop variable in 3.X comprehension expressions.  
Unlike compression loop variables, though, this variable is *removed* after the __except__ blocks exits in 3.X. It does so because it would otherwise retain a reference to the runtime call stack, which would defer garbage collection and thus retain excess memory space. This removal occurs, though, even if you're using the name elsewhere, and is more extreme policy than that used for comprehensions:

In [9]:
# If you need to reference the exception instance after the try statement, assign it to another name
try:
    1 / 0
except Exception as X:
    print(X)
    Saveit = X
Saveit
X

division by zero


ZeroDivisionError('division by zero')

NameError: name 'X' is not defined

* Ptyhon 3.X Exception Chaining: raise from

In [11]:
try:
    1 / 0
except Exception as E:
        raise TypeError('Bad') from E

TypeError: Bad

In [13]:
try:
    try:
        raise IndexError()
    except Exception as E:
        raise TypeError() from E
except Exception as E:
    raise SyntaxError() from E

SyntaxError: None (<string>)

## The assert Statement

In [ ]:
assert test, data      # The data part is optional

# works like the following code:
if __debug__:
    if not test:
        raise AssertionError(data)

In other words, if the __test__ evaluates to false, Python raises an exception: the __data__ item (if it's provided) is used as the exception's constructor argument.

As an added feature, __assert__ statements may be removed from a complied program's byte code if the __-0__ Python command-line flag is used, thereby optimizing the program. __AssertionError__ is a bulit-in exception, and the __\_\_debug\_\___ is a bulit-in name that is automatically set to __True__ unless the __-0__ flag is used. Use a command line like __python -0 main.py__ to run in optimized mode and disable (and hence skip) asserts.

In [4]:
def f(x):
    assert x < 0, 'x must be negative'
    return x ** 2
f(1)

AssertionError: x must be negative

## with/as Context Managers

with *expression* [as variable]:  
$\quad$with-block

In [ ]:
with open(r'C:\misc\data') as myfile:
    for line in myfile:
        print(line)
        ...

In [ ]:
# the decimal module also uses context managers to simplify saving and restroing the current
# decimal context, which specifies the precision and rounding characteristics for calculations:
import decimal
with decimal.localcontext() as ctx:
    ctx.prec = 2
    x = decimal.Decimal('1.00') / decimal.Decimal('3.00')

Here's how the __with__ statement actually works:  
* The expression is evaluated, resulting in an object known as a *context manger* that must have __\_\_enter\_\___ and __\_\_exit\_\___ methods.
* The context manager's __\_\_enter\_\___ method is called. The value it returns is assigned to the variable in the __as__ clause if present, or simply discarded otherwise.
* The code in the nested __with__ block is executed.
* If the __with__ block raises an exception, the __\_\_exit\_\_(type, value, traceback)__ method is called with the exception details. These are the same three values returned by __sys.exc_info__, described in the Python manuals and later in this part of the book. If this method returns a false value, the exception is reraised; otherwise, the exception is terminated. The exception should normally be reraised so that it is propagated outside the __with__ statement.
* If the __with__ block does not raise an exception, the __\_\_exit\_\___ method is still called, but its __type, value, traceback__ arguments are passed in as __None__.

In [4]:
class TraceBlock:
    def message(self, arg):
        print('running ' + arg)
    def __enter__(self):
        print('starting with block')
        return self
    def __exit__(self, exc_type, exc_value, exc_tb):
        if exc_type is None:
            print('exited normally\n')
        else:
            print('raise an exception! ' + str(exc_type))
            return False   # Propagate
if __name__ == '__main__':
    with TraceBlock() as action:
        action.message('test 1')
        print('reached')
    with TraceBlock() as action:
        action.message('test 2')
        raise TypeError
        print('not reached')

starting with block
running test 1
reached
exited normally

starting with block
running test 2
raise an exception! <class 'TypeError'>


TypeError: 

* Multiple Context Managers in 3.1, 2.7, and later

In [ ]:
with A() as a, B() as b:
    ...statements...
# is equivalent to the following:
with A() as a:
    with B() as b:
        ...statements...

In [ ]:
with open('data') as fin, open('res', 'w') as fout:
    for line in fin:
        if 'some key' in line:
            fout.write(line)
            
# To implement a parallel lines scan of two files
with open('script1.py') as f1, open('script2.py') as f2:
    for pair in zip(f1, f2):
        print(pair)
        
# To do a line-by-line comparison of two text files
with open('script1.py') as f1, open('script2.py') as f2:
    for (linenum, (line1, line2)) in enumerate(zip(line1, line2)):
        if line1 != line2:
            print('%s\n%r\n%r' % (linenum, line1, line2))

# Exception Objects

* Class exception are matched by *superclass relationships*: the raised exception matches an __except__ clause if that __except__ clause names the exception instance's class or any superclass of it.
* If we list a class name without paretheses in a __raise__, Python calls the class with no constructor argument to make an instance for us. Exception instances can be created before the __raise__, as done before, or within the __raise__ statement itself.

In [18]:
I = IndexError('spam')
I.args
print(I)
raise I

('spam',)

spam


IndexError: spam

In [20]:
class E(Exception): pass
try:
    raise E('spam')
except E as X:
    print(X)
    print(X.args)
    print(repr(X))
    
try:
    raise E('spam', 'eggs', 'ham')
except E as X:
    print('%s %s' % (X, X.args))

spam
('spam',)
E('spam')
('spam', 'eggs', 'ham') ('spam', 'eggs', 'ham')


In [21]:
class Mybad(Exception):
    def __str__(self):
        return 'Always look on the bright side of life ...'
try:
    raise Mybad()
except Mybad as X:
    print(X)
    
raise Mybad()

Always look on the bright side of life ...


Mybad: Always look on the bright side of life ...

In [ ]:
class FormatError(Exception):
    logfile = 'formaterror.py'
    def __init__(self, line, file):
        self.line = line
        self.file = file
    def logerror(self):
        log = open(self.logfile, 'a')
        print('Error at:', self.file, self.line, file=log)
def parser():
    raise FormatError(40, 'spam.txt')

if __name__ == '__main__':
    try:
        parser()
    except FormatError as exc:
        exc.logerror()

In [10]:
from jiayuvsyiyun import game

In [5]:
game()

jiayu win!
yiyun win!
jiayu win!
yiyun win!
yiyun win!
yiyun has longer legs than jiayu's!
jiayu win!
jiayu win!
jiayu win!
yiyun win!
jiayu win!
jiayu has longer legs than yiyun's!
yiyun win!
yiyun win!
yiyun win!
yiyun win!
yiyun win!
yiyun has longer legs than jiayu's!
jiayu win!
jiayu win!
yiyun win!
yiyun win!
yiyun win!
yiyun has longer legs than jiayu's!
jiayu win!
yiyun win!
jiayu win!
yiyun win!
yiyun win!
yiyun has longer legs than jiayu's!
yiyun win!
jiayu win!
jiayu win!
yiyun win!
yiyun win!
yiyun has longer legs than jiayu's!
yiyun win!
jiayu win!
yiyun win!
jiayu win!
jiayu win!
jiayu has longer legs than yiyun's!
yiyun win!
jiayu win!
jiayu win!
jiayu win!
yiyun win!
jiayu has longer legs than yiyun's!
yiyun win!
yiyun win!
yiyun win!
yiyun win!
yiyun win!
yiyun has longer legs than jiayu's!
jiayu win!
yiyun win!
yiyun win!
yiyun win!
jiayu win!
yiyun has longer legs than jiayu's!
yiyun win!
yiyun win!
yiyun win!
jiayu win!
yiyun win!
yiyun has longer legs than jiayu's!

3

In [6]:
game()

jiayu win!
jiayu win!
yiyun win!
yiyun win!
yiyun win!
yiyun has longer legs than jiayu's!
yiyun win!
jiayu win!
jiayu win!
yiyun win!
yiyun win!
yiyun has longer legs than jiayu's!
yiyun win!
jiayu win!
jiayu win!
jiayu win!
jiayu win!
jiayu has longer legs than yiyun's!
yiyun win!
jiayu win!
jiayu win!
jiayu win!
yiyun win!
jiayu has longer legs than yiyun's!
yiyun win!
jiayu win!
jiayu win!
yiyun win!
yiyun win!
yiyun has longer legs than jiayu's!
jiayu win!
yiyun win!
jiayu win!
yiyun win!
jiayu win!
jiayu has longer legs than yiyun's!
yiyun win!
yiyun win!
jiayu win!
yiyun win!
yiyun win!
yiyun has longer legs than jiayu's!
yiyun win!
jiayu win!
yiyun win!
jiayu win!
jiayu win!
jiayu has longer legs than yiyun's!
jiayu win!
yiyun win!
yiyun win!
yiyun win!
yiyun win!
yiyun has longer legs than jiayu's!
yiyun win!
yiyun win!
jiayu win!
jiayu win!
jiayu win!
jiayu has longer legs than yiyun's!
yiyun win!
jiayu win!
jiayu win!
yiyun win!
yiyun win!
yiyun has longer legs than jiayu's!

5

In [7]:
game()

jiayu win!
yiyun win!
jiayu win!
jiayu win!
jiayu win!
jiayu has longer legs than yiyun's!
jiayu win!
yiyun win!
jiayu win!
jiayu win!
yiyun win!
jiayu has longer legs than yiyun's!
jiayu win!
jiayu win!
jiayu win!
yiyun win!
yiyun win!
jiayu has longer legs than yiyun's!
jiayu win!
yiyun win!
yiyun win!
jiayu win!
jiayu win!
jiayu has longer legs than yiyun's!
yiyun win!
jiayu win!
jiayu win!
yiyun win!
yiyun win!
yiyun has longer legs than jiayu's!
jiayu win!
yiyun win!
jiayu win!
yiyun win!
jiayu win!
jiayu has longer legs than yiyun's!
yiyun win!
jiayu win!
jiayu win!
yiyun win!
yiyun win!
yiyun has longer legs than jiayu's!
yiyun win!
yiyun win!
jiayu win!
yiyun win!
yiyun win!
yiyun has longer legs than jiayu's!
yiyun win!
jiayu win!
yiyun win!
jiayu win!
yiyun win!
yiyun has longer legs than jiayu's!
jiayu win!
jiayu win!
yiyun win!
jiayu win!
yiyun win!
jiayu has longer legs than yiyun's!
jiayu win!
jiayu win!
jiayu win!
jiayu win!
yiyun win!
jiayu has longer legs than yiyun's!

7

In [8]:
game()

jiayu win!
yiyun win!
yiyun win!
jiayu win!
yiyun win!
yiyun has longer legs than jiayu's!
yiyun win!
jiayu win!
jiayu win!
jiayu win!
jiayu win!
jiayu has longer legs than yiyun's!
yiyun win!
jiayu win!
yiyun win!
jiayu win!
yiyun win!
yiyun has longer legs than jiayu's!
yiyun win!
yiyun win!
yiyun win!
jiayu win!
jiayu win!
yiyun has longer legs than jiayu's!
yiyun win!
jiayu win!
jiayu win!
yiyun win!
yiyun win!
yiyun has longer legs than jiayu's!
yiyun win!
jiayu win!
jiayu win!
jiayu win!
jiayu win!
jiayu has longer legs than yiyun's!
jiayu win!
jiayu win!
yiyun win!
yiyun win!
yiyun win!
yiyun has longer legs than jiayu's!
yiyun win!
jiayu win!
yiyun win!
jiayu win!
jiayu win!
jiayu has longer legs than yiyun's!
yiyun win!
yiyun win!
jiayu win!
jiayu win!
jiayu win!
jiayu has longer legs than yiyun's!
yiyun win!
yiyun win!
jiayu win!
jiayu win!
yiyun win!
yiyun has longer legs than jiayu's!
jiayu win!
yiyun win!
yiyun win!
jiayu win!
yiyun win!
yiyun has longer legs than jiayu's!

4

In [9]:
game()

jiayu win!
yiyun win!
yiyun win!
yiyun win!
yiyun win!
yiyun has longer legs than jiayu's!
jiayu win!
yiyun win!
yiyun win!
yiyun win!
jiayu win!
yiyun has longer legs than jiayu's!
yiyun win!
jiayu win!
yiyun win!
jiayu win!
jiayu win!
jiayu has longer legs than yiyun's!
yiyun win!
yiyun win!
yiyun win!
jiayu win!
yiyun win!
yiyun has longer legs than jiayu's!
yiyun win!
yiyun win!
yiyun win!
yiyun win!
jiayu win!
yiyun has longer legs than jiayu's!
yiyun win!
yiyun win!
jiayu win!
jiayu win!
yiyun win!
yiyun has longer legs than jiayu's!
yiyun win!
yiyun win!
yiyun win!
jiayu win!
jiayu win!
yiyun has longer legs than jiayu's!
yiyun win!
jiayu win!
yiyun win!
jiayu win!
jiayu win!
jiayu has longer legs than yiyun's!
yiyun win!
jiayu win!
yiyun win!
jiayu win!
jiayu win!
jiayu has longer legs than yiyun's!
jiayu win!
jiayu win!
jiayu win!
jiayu win!
yiyun win!
jiayu has longer legs than yiyun's!
yiyun win!
jiayu win!
yiyun win!
jiayu win!
jiayu win!
jiayu has longer legs than yiyun's!

5

In [3]:
game()

yiyun win!
yiyun win!
jiayu win!
jiayu win!
yiyun win!
yiyun has longer legs than jiayu's!
jiayu win!
jiayu win!
yiyun win!
jiayu win!
yiyun win!
jiayu has longer legs than yiyun's!
yiyun win!
yiyun win!
jiayu win!
yiyun win!
jiayu win!
yiyun has longer legs than jiayu's!
jiayu win!
jiayu win!
yiyun win!
yiyun win!
jiayu win!
jiayu has longer legs than yiyun's!
yiyun win!
yiyun win!
jiayu win!
jiayu win!
yiyun win!
yiyun has longer legs than jiayu's!
yiyun win!
jiayu win!
jiayu win!
yiyun win!
jiayu win!
jiayu has longer legs than yiyun's!
jiayu win!
yiyun win!
yiyun win!
jiayu win!
yiyun win!
yiyun has longer legs than jiayu's!
jiayu win!
jiayu win!
jiayu win!
yiyun win!
jiayu win!
jiayu has longer legs than yiyun's!
yiyun win!
yiyun win!
yiyun win!
jiayu win!
yiyun win!
yiyun has longer legs than jiayu's!
yiyun win!
yiyun win!
jiayu win!
yiyun win!
yiyun win!
yiyun has longer legs than jiayu's!
jiayu win!
yiyun win!
yiyun win!
yiyun win!
jiayu win!
yiyun has longer legs than jiayu's!

4

In [4]:
game()

yiyun win!
yiyun win!
jiayu win!
yiyun win!
yiyun win!
yiyun has longer legs than jiayu's!
jiayu win!
yiyun win!
yiyun win!
yiyun win!
yiyun win!
yiyun has longer legs than jiayu's!
yiyun win!
yiyun win!
jiayu win!
yiyun win!
yiyun win!
yiyun has longer legs than jiayu's!
yiyun win!
jiayu win!
yiyun win!
yiyun win!
yiyun win!
yiyun has longer legs than jiayu's!
yiyun win!
jiayu win!
jiayu win!
jiayu win!
jiayu win!
jiayu has longer legs than yiyun's!
jiayu win!
yiyun win!
yiyun win!
yiyun win!
jiayu win!
yiyun has longer legs than jiayu's!
jiayu win!
jiayu win!
jiayu win!
yiyun win!
jiayu win!
jiayu has longer legs than yiyun's!
yiyun win!
jiayu win!
jiayu win!
jiayu win!
jiayu win!
jiayu has longer legs than yiyun's!
yiyun win!
jiayu win!
yiyun win!
jiayu win!
yiyun win!
yiyun has longer legs than jiayu's!
jiayu win!
jiayu win!
yiyun win!
yiyun win!
jiayu win!
jiayu has longer legs than yiyun's!
jiayu win!
yiyun win!
yiyun win!
yiyun win!
jiayu win!
yiyun has longer legs than jiayu's!

4

In [3]:
game()

yiyun win!
yiyun win!
yiyun win!
jiayu win!
yiyun win!
yiyun has longer legs than jiayu's!
jiayu win!
yiyun win!
yiyun win!
yiyun win!
yiyun win!
yiyun has longer legs than jiayu's!
yiyun win!
jiayu win!
jiayu win!
yiyun win!
jiayu win!
jiayu has longer legs than yiyun's!
yiyun win!
jiayu win!
jiayu win!
jiayu win!
jiayu win!
jiayu has longer legs than yiyun's!
jiayu win!
yiyun win!
jiayu win!
yiyun win!
yiyun win!
yiyun has longer legs than jiayu's!
jiayu win!
jiayu win!
yiyun win!
jiayu win!
yiyun win!
jiayu has longer legs than yiyun's!
yiyun win!
jiayu win!
jiayu win!
jiayu win!
jiayu win!
jiayu has longer legs than yiyun's!
jiayu win!
yiyun win!
yiyun win!
yiyun win!
yiyun win!
yiyun has longer legs than jiayu's!
jiayu win!
jiayu win!
yiyun win!
yiyun win!
jiayu win!
jiayu has longer legs than yiyun's!
jiayu win!
jiayu win!
jiayu win!
jiayu win!
jiayu win!
jiayu has longer legs than yiyun's!
jiayu win!
jiayu win!
yiyun win!
yiyun win!
jiayu win!
jiayu has longer legs than yiyun's!

7

In [11]:
game()

jiayu win!
yiyun win!
yiyun win!
yiyun win!
jiayu win!
yiyun has longer legs than jiayu's!
yiyun win!
jiayu win!
yiyun win!
yiyun win!
yiyun win!
yiyun has longer legs than jiayu's!
jiayu win!
jiayu win!
jiayu win!
yiyun win!
jiayu win!
jiayu has longer legs than yiyun's!
yiyun win!
yiyun win!
yiyun win!
yiyun win!
jiayu win!
yiyun has longer legs than jiayu's!
jiayu win!
yiyun win!
jiayu win!
yiyun win!
yiyun win!
yiyun has longer legs than jiayu's!
jiayu win!
jiayu win!
yiyun win!
yiyun win!
jiayu win!
jiayu has longer legs than yiyun's!
jiayu win!
jiayu win!
jiayu win!
yiyun win!
yiyun win!
jiayu has longer legs than yiyun's!
jiayu win!
jiayu win!
jiayu win!
jiayu win!
jiayu win!
jiayu has longer legs than yiyun's!
jiayu win!
yiyun win!
jiayu win!
yiyun win!
jiayu win!
jiayu has longer legs than yiyun's!
yiyun win!
yiyun win!
jiayu win!
jiayu win!
jiayu win!
jiayu has longer legs than yiyun's!
jiayu win!
yiyun win!
jiayu win!
jiayu win!
yiyun win!
jiayu has longer legs than yiyun's!

7

In [12]:
game()

jiayu win!
jiayu win!
jiayu win!
jiayu win!
jiayu win!
jiayu has longer legs than yiyun's!
yiyun win!
yiyun win!
yiyun win!
jiayu win!
yiyun win!
yiyun has longer legs than jiayu's!
yiyun win!
yiyun win!
yiyun win!
yiyun win!
yiyun win!
yiyun has longer legs than jiayu's!
jiayu win!
jiayu win!
yiyun win!
yiyun win!
yiyun win!
yiyun has longer legs than jiayu's!
jiayu win!
jiayu win!
jiayu win!
yiyun win!
jiayu win!
jiayu has longer legs than yiyun's!
yiyun win!
yiyun win!
jiayu win!
yiyun win!
jiayu win!
yiyun has longer legs than jiayu's!
jiayu win!
yiyun win!
jiayu win!
yiyun win!
yiyun win!
yiyun has longer legs than jiayu's!
yiyun win!
jiayu win!
yiyun win!
yiyun win!
yiyun win!
yiyun has longer legs than jiayu's!
jiayu win!
jiayu win!
yiyun win!
jiayu win!
yiyun win!
jiayu has longer legs than yiyun's!
jiayu win!
jiayu win!
yiyun win!
yiyun win!
jiayu win!
jiayu has longer legs than yiyun's!
yiyun win!
yiyun win!
jiayu win!
yiyun win!
yiyun win!
yiyun has longer legs than jiayu's!

4

In [13]:
game()

jiayu win!
yiyun win!
yiyun win!
yiyun win!
jiayu win!
yiyun has longer legs than jiayu's!
jiayu win!
jiayu win!
yiyun win!
jiayu win!
jiayu win!
jiayu has longer legs than yiyun's!
yiyun win!
jiayu win!
yiyun win!
yiyun win!
yiyun win!
yiyun has longer legs than jiayu's!
jiayu win!
yiyun win!
jiayu win!
jiayu win!
jiayu win!
jiayu has longer legs than yiyun's!
jiayu win!
jiayu win!
jiayu win!
yiyun win!
jiayu win!
jiayu has longer legs than yiyun's!
jiayu win!
yiyun win!
yiyun win!
jiayu win!
jiayu win!
jiayu has longer legs than yiyun's!
yiyun win!
yiyun win!
yiyun win!
yiyun win!
jiayu win!
yiyun has longer legs than jiayu's!
yiyun win!
yiyun win!
jiayu win!
yiyun win!
jiayu win!
yiyun has longer legs than jiayu's!
jiayu win!
yiyun win!
yiyun win!
yiyun win!
yiyun win!
yiyun has longer legs than jiayu's!
jiayu win!
yiyun win!
jiayu win!
jiayu win!
jiayu win!
jiayu has longer legs than yiyun's!
jiayu win!
jiayu win!
jiayu win!
yiyun win!
yiyun win!
jiayu has longer legs than yiyun's!

6

In [14]:
game()

yiyun win!
jiayu win!
jiayu win!
jiayu win!
jiayu win!
jiayu has longer legs than yiyun's!
jiayu win!
yiyun win!
yiyun win!
jiayu win!
jiayu win!
jiayu has longer legs than yiyun's!
yiyun win!
yiyun win!
jiayu win!
jiayu win!
jiayu win!
jiayu has longer legs than yiyun's!
yiyun win!
yiyun win!
yiyun win!
yiyun win!
jiayu win!
yiyun has longer legs than jiayu's!
yiyun win!
jiayu win!
yiyun win!
jiayu win!
jiayu win!
jiayu has longer legs than yiyun's!
jiayu win!
yiyun win!
jiayu win!
yiyun win!
jiayu win!
jiayu has longer legs than yiyun's!
jiayu win!
jiayu win!
yiyun win!
yiyun win!
jiayu win!
jiayu has longer legs than yiyun's!
jiayu win!
yiyun win!
jiayu win!
jiayu win!
yiyun win!
jiayu has longer legs than yiyun's!
yiyun win!
jiayu win!
yiyun win!
yiyun win!
jiayu win!
yiyun has longer legs than jiayu's!
yiyun win!
yiyun win!
jiayu win!
yiyun win!
yiyun win!
yiyun has longer legs than jiayu's!
jiayu win!
jiayu win!
jiayu win!
yiyun win!
yiyun win!
jiayu has longer legs than yiyun's!

8